#A* Algorithm

In [2]:
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import time

# ---------- Heuristic: Misplaced Tiles ----------
def misplaced_tiles(puzzle, goal):
    return sum(1 for i in range(len(puzzle)) if puzzle[i] != goal[i] and puzzle[i] != 0)

# ---------- Print Puzzle ----------
def print_puzzle(puzzle):
    for i in range(0, 9, 3):
        print(" ".join(str(x) if x != 0 else "_" for x in puzzle[i:i+3]))
    print("-" * 8)

# ---------- A* Solver ----------
def a_star_solver(puzzle, goal):
    steps = np.array([
        ('up', [0, 1, 2], -3),
        ('down', [6, 7, 8], 3),
        ('left', [0, 3, 6], -1),
        ('right', [2, 5, 8], 1)
    ], dtype=[('move', 'U5'), ('head', list), ('shift', int)])

    dtstate = [('puzzle', list), ('parent', int), ('gn', int), ('hn', int)]
    state = np.array([(puzzle, -1, 0, misplaced_tiles(puzzle, goal))], dtstate)
    dtpriority = [('position', int), ('fn', int)]
    priority = np.array([(0, misplaced_tiles(puzzle, goal))], dtpriority)

    while True:
        priority = np.sort(priority, kind='mergesort', order=['fn', 'position'])
        position = priority[0][0]
        fn = priority[0][1]
        priority = np.delete(priority, 0, 0)

        puzzle, parent, gn, hn = state[position]
        puzzle = np.array(puzzle)
        blank = int(np.where(puzzle == 0)[0])
        gn = gn + 1

        for move in steps:
            if blank in move['head']:
                continue
            temp = puzzle.copy()
            swap = blank + move['shift']
            temp[blank], temp[swap] = temp[swap], temp[blank]

            if any((temp == x['puzzle']).all() for x in state):
                continue

            hn = misplaced_tiles(temp, goal)
            fn = gn + hn
            new_state = np.array([(temp.tolist(), position, gn, hn)], dtstate)
            state = np.append(state, new_state)
            new_priority = np.array([(len(state) - 1, fn)], dtpriority)
            priority = np.append(priority, new_priority)

            if temp.tolist() == goal:
                # reconstruct path
                path = [temp.tolist()]
                p = position
                while p != -1:
                    path.insert(0, state[p]['puzzle'])
                    p = state[p]['parent']
                return path

# ---------- Game Class ----------
class PuzzleGame:
    def __init__(self):
        self.puzzle = [2, 8, 3, 1, 6, 4, 7, 0, 5]
        self.goal = [1, 2, 3, 8, 0, 4, 7, 6, 5]
        self.create_ui()
        self.show_puzzle()

    def create_ui(self):
        self.up_btn = widgets.Button(description='↑ Up', button_style='info')
        self.down_btn = widgets.Button(description='↓ Down', button_style='info')
        self.left_btn = widgets.Button(description='← Left', button_style='info')
        self.right_btn = widgets.Button(description='→ Right', button_style='info')
        self.solve_btn = widgets.Button(description='🤖 Solve with A*', button_style='success')
        self.reset_btn = widgets.Button(description='🔁 Reset', button_style='warning')

        self.up_btn.on_click(lambda x: self.move('up'))
        self.down_btn.on_click(lambda x: self.move('down'))
        self.left_btn.on_click(lambda x: self.move('left'))
        self.right_btn.on_click(lambda x: self.move('right'))
        self.solve_btn.on_click(lambda x: self.solve())
        self.reset_btn.on_click(lambda x: self.reset())

        display(widgets.HBox([self.up_btn, self.down_btn, self.left_btn, self.right_btn]))
        display(widgets.HBox([self.solve_btn, self.reset_btn]))

    def show_puzzle(self):
        clear_output(wait=True)
        display(widgets.HTML("<h3>🎯 8-Puzzle Game (A* Search)</h3>"))
        for i in range(0, 9, 3):
            row = " ".join(str(x) if x != 0 else "_" for x in self.puzzle[i:i+3])
            print(row)
        print("-" * 8)
        display(widgets.HBox([self.up_btn, self.down_btn, self.left_btn, self.right_btn]))
        display(widgets.HBox([self.solve_btn, self.reset_btn]))

    def move(self, direction):
        blank = self.puzzle.index(0)
        moves = {'up': -3, 'down': 3, 'left': -1, 'right': 1}
        invalid = {'up': [0, 1, 2], 'down': [6, 7, 8], 'left': [0, 3, 6], 'right': [2, 5, 8]}

        if blank in invalid[direction]:
            print("🚫 Invalid Move!")
            return

        swap = blank + moves[direction]
        self.puzzle[blank], self.puzzle[swap] = self.puzzle[swap], self.puzzle[blank]
        self.show_puzzle()

    def solve(self):
        print("🤖 Solving with A* algorithm...")
        path = a_star_solver(self.puzzle, self.goal)
        for step in path:
            self.puzzle = step
            self.show_puzzle()
            time.sleep(0.5)
        print("✅ Goal Reached!")

    def reset(self):
        self.puzzle = [2, 8, 3, 1, 6, 4, 7, 0, 5]
        self.show_puzzle()

# ---------- Run the Game ----------
game = PuzzleGame()


HTML(value='<h3>🎯 8-Puzzle Game (A* Search)</h3>')

2 8 3
1 6 4
7 _ 5
--------
